## Working with google-clouds BIG query

#### We first require Requests, which will be used to import our data from the .json feed into Python allowing for transformation using Pandas. You should notice however that we with Pandas, we actually import the entire library as well as the specific object json_normalize which is specifically designed to transform data from json objects into Dataframe objects.

#### Additional libraries that import are sys, datetime, and gc. sys is being used to call a system function that will help us stop Python from continuing in the case when certain criteria are met within our ETL. datetime is being used to transform datetime objects provided by the json API. Lastly, garbage collection, or gc is being used to clean up the memory footprint of our machine as we run our very basic ETL as a catch all to protect our laptop in case for some reason the script does not end as expected.

In [27]:
import requests
import pandas as pd
import sys
from pandas.io.json import json_normalize
from datetime import datetime
import gc
from google.oauth2 import credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.cloud import bigquery

### Lastly, for connecting to BigQuery, we need to install pandas-gbq in our Python environment so that it is available for Pandas to use later in this post.

In [ ]:
!pip install pandas-gbq -U

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
!pip install --upgrade google-auth

### Following Google API : https://googleapis.github.io/google-api-python-client/docs/oauth.html

In [28]:
# Set the scopes for the desired permissions
SCOPES = ['https://www.googleapis.com/auth/bigquery']

# Create the flow for authorization
flow = InstalledAppFlow.from_client_secrets_file(
    '...', scopes=SCOPES
)
credentials = flow.run_local_server(port=0)

project_id = '...'

# Use the obtained credentials for authentication
client = bigquery.Client(credentials=credentials, project=project_id) #

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=853561264369-cg46opisk56diiv2ee4nblkb6jmtland.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57279%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=iXhNJRSkVBXH4zhU6Ws8Va5h0yEytY&access_type=offline


### We'll work with a dataset of posts on Hacker News, a website focusing on computer science and cybersecurity news.

### In BigQuery, each dataset is contained in a corresponding project. In this case, our hacker_news dataset is contained in the bigquery-public-data project. To access the dataset,

We begin by constructing a reference to the dataset with the dataset() method.
Next, we use the get_dataset() method, along with the reference we just constructed, to fetch the dataset.

In [29]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

### list_tables() method to list the tables in the dataset.

In [30]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


### Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the full table in the hacker_news dataset.

In [31]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

## Table schema:

### The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.
In this example, we'll investigate the full table that we fetched above.
Each SchemaField tells us about a specific column (which we also refer to as a field). In order, the information is:

The name of the column;

The field type (or datatype) in the column;

The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default);

A description of the data in that column;

In [32]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABL

### We can use the list_rows() method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery RowIterator object that can quickly be converted to a pandas DataFrame with the to_dataframe() method.

In [33]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"I would rather just have wired earbuds, period...",<NA>,zeveb,<NA>,1591717736,2020-06-09 15:48:56+00:00,comment,23467666,23456782,<NA>,<NA>,<NA>
1,None,None,DNS?,<NA>,nly,<NA>,1572810465,2019-11-03 19:47:45+00:00,comment,21436112,21435130,<NA>,<NA>,<NA>
2,None,None,These benchmarks seem pretty good. Filterable...,<NA>,mrkeen,<NA>,1591717727,2020-06-09 15:48:47+00:00,comment,23467665,23467426,<NA>,<NA>,<NA>
3,None,None,Oh really?<p>* Excel alone uses 86.1MB of priv...,<NA>,oceanswave,<NA>,1462987532,2016-05-11 17:25:32+00:00,comment,11677248,11676886,<NA>,<NA>,<NA>
4,None,None,These systems are useless. Of the many flaws:...,<NA>,nyxxie,<NA>,1572810473,2019-11-03 19:47:53+00:00,comment,21436113,21435025,<NA>,<NA>,<NA>


In [34]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


## Fun Analysis with SQL

#### Query that returns all authors with more than 10,000 posts as well as their post counts

In [35]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,9734136,None,None,1434565400,2015-06-17 18:23:20+00:00,None,9733698,True,<NA>,0
1,4921158,None,None,1355496966,2012-12-14 14:56:06+00:00,None,4921100,True,<NA>,0
2,7500568,None,None,1396261158,2014-03-31 10:19:18+00:00,None,7499385,True,<NA>,0
3,8909635,None,None,1421627275,2015-01-19 00:27:55+00:00,None,8901135,True,<NA>,0
4,9256463,None,None,1427204705,2015-03-24 13:45:05+00:00,None,9256346,True,<NA>,0


#### The COUNT(1) function replaces all records from the query result set with value 1. If you have NULL values, it is also replaced by 1. Therefore, COUNT(1) also returns the total number of records (including NULLs) in the table.

Since we have some nulls in author I will have this as count(1) to count all the records

In [50]:
query1 = """
        SELECT author, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY author
        HAVING COUNT(1) > 10000
        """

In [51]:
query_job2 = client.query(query1)

In [52]:
posts_author = query_job2.to_dataframe()

In [53]:
posts_author

,author,NumPosts
0,DanBC,12902
1,davidw,10764
2,eru,10448
3,jacquesm,21107
4,sp332,10882
5,rayiner,11080
6,rbanffy,10557
7,None,227736
8,tptacek,33839
9,anigbrowl,11395


## Deleted comments

#### How many comments have been deleted? (If a comment was deleted, the `deleted` column in the comments table will have the value `True`.)

In [63]:
query2 = """
        SELECT COUNT(1) as deleted_total
        FROM `bigquery-public-data.hacker_news.comments`
        WHERE deleted = True
        """

In [64]:
query_job3 = client.query(query2)

In [65]:
deleted_comments = query_job3.to_dataframe()
deleted_comments

,deleted_total
0,227736
